In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# --- 1. 配置 ---
# 输入文件夹 (包含 _selected.csv 文件)
INPUT_FOLDER = 'Feature_Selected_Data' 
# 输出文件夹 (存放本次 OLS 预测结果)
OUTPUT_FOLDER = 'OLS_Prediction_Result' 
# 输入文件名
TRAIN_PRICE_FILE = 'train_price_selected.csv'
TEST_PRICE_FILE = 'test_price_selected.csv'
# 输出文件名
OUTPUT_FILE = 'submission_ols_selected_features.csv'

TARGET_COLUMN = 'Price'
ID_COLUMN = 'ID'

# --- 2. 创建输出文件夹 ---
try:
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"输出文件夹 '{OUTPUT_FOLDER}' 已创建或已存在。")
except OSError as e:
    print(f"创建文件夹 '{OUTPUT_FOLDER}' 时出错: {e}")
    exit()

# --- 3. 加载经过特征筛选的数据 ---
print(f"\n--- 从 '{INPUT_FOLDER}' 加载 Price 数据 ---")
try:
    train_df = pd.read_csv(os.path.join(INPUT_FOLDER, TRAIN_PRICE_FILE), encoding='utf-8-sig')
    test_df = pd.read_csv(os.path.join(INPUT_FOLDER, TEST_PRICE_FILE), encoding='utf-8-sig')
    print("Price 数据加载成功。")
    print(f"  训练集形状: {train_df.shape}")
    print(f"  测试集形状: {test_df.shape}")
except FileNotFoundError as e:
    print(f"加载数据时出错: {e}. 请确保文件路径正确。")
    exit()
except Exception as e:
    print(f"加载数据时发生其他错误: {e}")
    exit()

# 存储测试集的 ID
test_ids = test_df[ID_COLUMN].copy()

# --- 4. 准备数据 (分离特征和目标, 对齐) ---
print("\n--- 准备训练和测试数据 ---")

try:
    # 分离目标变量
    y_train = train_df[TARGET_COLUMN]
    X_train = train_df.drop(columns=[TARGET_COLUMN])
    
    # 从测试集移除 ID 列
    X_test = test_df.drop(columns=[ID_COLUMN])

    # 识别并移除可能残余的非数值列 (保险起见)
    non_numeric_train = X_train.select_dtypes(exclude=np.number).columns
    non_numeric_test = X_test.select_dtypes(exclude=np.number).columns
    if not non_numeric_train.empty:
        print(f"  警告: 训练集发现非数值列，将移除: {non_numeric_train.tolist()}")
        X_train = X_train.drop(columns=non_numeric_train)
    if not non_numeric_test.empty:
        print(f"  警告: 测试集发现非数值列，将移除: {non_numeric_test.tolist()}")
        X_test = X_test.drop(columns=non_numeric_test)

    # 对齐列 (非常重要)
    train_cols = X_train.columns
    test_cols = X_test.columns

    missing_in_test = set(train_cols) - set(test_cols)
    for c in missing_in_test:
        X_test[c] = 0 # 添加缺失列并填充 0

    extra_in_test = set(test_cols) - set(train_cols)
    if extra_in_test:
        print(f"  警告: 测试集存在额外列，将移除: {list(extra_in_test)}")
        X_test = X_test.drop(columns=list(extra_in_test))

    # 确保列顺序一致
    X_test = X_test[train_cols]
    
    print(f"  数据准备完成。训练特征形状: {X_train.shape}, 测试特征形状: {X_test.shape}")

except KeyError as e:
    print(f"数据准备时出错: 找不到列 {e}。请检查列名配置。")
    exit()
except Exception as e:
     print(f"数据准备时发生未知错误: {e}")
     exit()

# --- 5. 定义并训练 OLS Pipeline ---
print("\n--- 定义并训练 OLS 模型 Pipeline ---")

ols_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), # 处理可能存在的 NaN
    ('scaler', StandardScaler()),                 # 特征缩放
    ('model', LinearRegression(n_jobs=-1))        # OLS 模型, n_jobs=-1 使用所有 CPU
])

try:
    print("  开始训练...")
    ols_pipeline.fit(X_train, y_train)
    print("  OLS 模型训练完成。")
except Exception as e:
    print(f"  训练 OLS 模型时出错: {e}")
    exit()
    
# --- 6. 在训练集上评估模型 ---
print("\n--- 在完整训练集上评估 OLS 模型 ---")
try:
    y_train_pred = ols_pipeline.predict(X_train)
    
    # 计算指标
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    mae_train = mean_absolute_error(y_train, y_train_pred)
    medae_train = median_absolute_error(y_train, y_train_pred)
    
    print(f"  训练集 RMSE:  {rmse_train:.4f}")
    print(f"  训练集 MAE:    {mae_train:.4f}")
    print(f"  训练集 MedAE: {medae_train:.4f}")
    
except Exception as e:
    print(f"  评估训练集时出错: {e}")

# --- 7. 在测试集上进行预测 ---
print("\n--- 在测试集上进行预测 ---")
try:
    predictions = ols_pipeline.predict(X_test)
    print("  预测完成。")
    
    # 可选: 确保预测值为非负数
    # predictions = np.clip(predictions, a_min=0, a_max=None)
    # print("  已将负数预测值修正为 0。")

except Exception as e:
    print(f"  在测试集上预测时出错: {e}")
    exit()

# # --- 8. 创建并保存提交文件 ---
# print(f"\n--- 创建并保存提交文件到 '{OUTPUT_FOLDER}' ---")

# submission_df = pd.DataFrame({
#     ID_COLUMN: test_ids,
#     'PredictedPrice': predictions # 使用预测结果
# })

# # 确保 ID 是整数
# submission_df[ID_COLUMN] = submission_df[ID_COLUMN].astype(int)

# # 按 ID 排序 (可选但推荐)
# submission_df = submission_df.sort_values(by=ID_COLUMN)

# try:
#     output_path = os.path.join(OUTPUT_FOLDER, OUTPUT_FILE)
#     submission_df.to_csv(output_path, index=False, encoding='utf-8-sig')
#     print(f"✓ 提交文件已成功保存: {output_path}")
#     print(f"  总预测条数: {len(submission_df)}")
#     print("\n提交文件预览 (前5行):")
#     print(submission_df.head())
# except Exception as e:
#     print(f"✗ 保存提交文件时出错: {e}")

print("\n--- OLS 预测流程完成 ---")

输出文件夹 'OLS_Prediction_Result' 已创建或已存在。

--- 从 'Feature_Selected_Data' 加载 Price 数据 ---
Price 数据加载成功。
  训练集形状: (103871, 155)
  测试集形状: (34017, 155)

--- 准备训练和测试数据 ---
  警告: 训练集发现非数值列，将移除: ['楼层_中楼层', '楼层_低楼层', '楼层_地下室', '楼层_底层', '楼层_顶层', '楼层_高楼层', '环线_三至四环', '环线_中环至外环', '环线_二环内', '环线_二至三环', '环线_五至六环', '环线_六环外', '环线_内环内', '环线_内环至中环', '环线_内环至外环', '环线_四至五环', '环线_外环外', '环线_无环线', '电梯_无', '电梯_有', '电梯_未知', '建筑结构_未知结构', '建筑结构_框架结构', '建筑结构_混合结构', '建筑结构_砖木结构', '建筑结构_砖混结构', '建筑结构_钢混结构', '建筑结构_钢结构', '装修_精装', '装修_简装', '装修_毛坯', '装修_其他', '装修_未知', '装修_精装.1', '装修_简装.1', '装修_毛坯.1', '装修_其他.1', '装修_未知.1', '别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排', '别墅_非别墅', '产权所属_共有', '产权所属_非共有']
  警告: 测试集发现非数值列，将移除: ['楼层_中楼层', '楼层_低楼层', '楼层_地下室', '楼层_底层', '楼层_顶层', '楼层_高楼层', '环线_三至四环', '环线_中环至外环', '环线_二环内', '环线_二至三环', '环线_五至六环', '环线_六环外', '环线_内环内', '环线_内环至中环', '环线_内环至外环', '环线_四至五环', '环线_外环外', '环线_无环线', '电梯_无', '电梯_有', '电梯_未知', '建筑结构_未知结构', '建筑结构_框架结构', '建筑结构_混合结构', '建筑结构_砖木结构', '建筑结构_砖混结构', '建筑结构_钢混结构', '建筑结构_钢结构', '装修_精装', '装修_简装',

In [2]:
import pandas as pd
import numpy as np
import os
import joblib # 用于将来可能保存训练好的模型/缩放器
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import Lasso # 导入 Lasso
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, median_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# --- 1. 配置 ---
# 输入文件夹 (包含 _selected.csv 文件)
INPUT_FOLDER = 'Feature_Selected_Data' 
# 输出文件夹 (存放本次 Lasso 预测结果)
OUTPUT_FOLDER = 'Lasso_Prediction_Result' 
# 输入文件名
TRAIN_PRICE_FILE = 'train_price_selected.csv'
TEST_PRICE_FILE = 'test_price_selected.csv'
# 输出文件名
OUTPUT_FILE = 'submission_lasso_6fold_mae.csv'

TARGET_COLUMN = 'Price'
ID_COLUMN = 'ID'

# 交叉验证折数设置为 6
N_SPLITS = 6 
RANDOM_STATE = 42 # 保证结果可复现

# --- 2. 创建输出文件夹 ---
try:
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"输出文件夹 '{OUTPUT_FOLDER}' 已创建或已存在。")
except OSError as e:
    print(f"创建文件夹 '{OUTPUT_FOLDER}' 时出错: {e}")
    exit()

# --- 3. 加载经过特征筛选的数据 ---
print(f"\n--- 从 '{INPUT_FOLDER}' 加载 Price 数据 ---")
try:
    train_df = pd.read_csv(os.path.join(INPUT_FOLDER, TRAIN_PRICE_FILE), encoding='utf-8-sig')
    test_df = pd.read_csv(os.path.join(INPUT_FOLDER, TEST_PRICE_FILE), encoding='utf-8-sig')
    print("Price 数据加载成功。")
    print(f"  训练集形状: {train_df.shape}")
    print(f"  测试集形状: {test_df.shape}")
except FileNotFoundError as e:
    print(f"加载数据时出错: {e}. 请确保文件路径正确。")
    exit()
except Exception as e:
    print(f"加载数据时发生其他错误: {e}")
    exit()

# 存储测试集的 ID
test_ids = test_df[ID_COLUMN].copy()

# --- 4. 准备数据 (分离特征和目标, 对齐) ---
print("\n--- 准备训练和测试数据 ---")
try:
    # 分离目标变量
    y_train = train_df[TARGET_COLUMN]
    X_train = train_df.drop(columns=[TARGET_COLUMN])
    
    # 从测试集移除 ID 列
    X_test = test_df.drop(columns=[ID_COLUMN])

    # 识别并移除可能残余的非数值列 (保险起见)
    non_numeric_train = X_train.select_dtypes(exclude=np.number).columns
    non_numeric_test = X_test.select_dtypes(exclude=np.number).columns
    if not non_numeric_train.empty:
        print(f"  警告: 训练集发现非数值列，将移除: {non_numeric_train.tolist()}")
        X_train = X_train.drop(columns=non_numeric_train)
    if not non_numeric_test.empty:
        print(f"  警告: 测试集发现非数值列，将移除: {non_numeric_test.tolist()}")
        X_test = X_test.drop(columns=non_numeric_test)

    # 对齐列 (非常重要)
    train_cols = X_train.columns
    test_cols = X_test.columns

    missing_in_test = set(train_cols) - set(test_cols)
    for c in missing_in_test:
        X_test[c] = 0 # 添加缺失列并填充 0

    extra_in_test = set(test_cols) - set(train_cols)
    if extra_in_test:
        print(f"  警告: 测试集存在额外列，将移除: {list(extra_in_test)}")
        X_test = X_test.drop(columns=list(extra_in_test))

    # 确保列顺序一致
    X_test = X_test[train_cols]
    
    print(f"  数据准备完成。训练特征形状: {X_train.shape}, 测试特征形状: {X_test.shape}")

except KeyError as e:
    print(f"数据准备时出错: 找不到列 {e}。请检查列名配置。")
    exit()
except Exception as e:
     print(f"数据准备时发生未知错误: {e}")
     exit()

# --- 5. 定义评价标准 (MAE) 和交叉验证策略 ---
print("\n--- 定义评价标准 (MAE) 和交叉验证策略 ---")
# 使用 MAE 作为评价标准，越小越好
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
print(f"  评价标准: MAE (越低越好)")

# 定义 6 折交叉验证
cv_strategy = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
print(f"  交叉验证策略: {N_SPLITS}-折 KFold")

# --- 6. 定义 Lasso Pipeline 和参数网格 ---
print("\n--- 定义 Lasso Pipeline 和参数网格 ---")

lasso_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), # 处理 NaN
    ('scaler', StandardScaler()),                 # 特征缩放
    ('model', Lasso(random_state=RANDOM_STATE, 
                    max_iter=3000)) # 增加最大迭代次数，Lasso 可能需要更多次迭代来收敛
])

# 定义 Lasso 的 alpha 参数搜索范围 (可以根据需要调整范围和数量)
param_grid_lasso = {'model__alpha': np.logspace(-4, 2, 7)} # 例如: [0.0001, 0.001, ..., 100]
print(f"  Lasso alpha 搜索范围: {param_grid_lasso['model__alpha']}")

# --- 7. 使用 GridSearchCV 进行训练和调优 ---
print("\n--- 运行 GridSearchCV (Lasso) ---")
print(f"  使用 {N_SPLITS}-折交叉验证优化 MAE...")

grid_lasso = GridSearchCV(
    estimator=lasso_pipeline, 
    param_grid=param_grid_lasso, 
    scoring=mae_scorer, 
    cv=cv_strategy, 
    n_jobs=-1,        # 使用所有 CPU 核心
    refit=True         # refit=True 会在找到最佳参数后，自动用全部训练数据重新训练模型
)

try:
    # 执行网格搜索和交叉验证
    grid_lasso.fit(X_train, y_train)
    
    print("\n  GridSearchCV 完成。")
    print(f"  最优 Lasso 交叉验证得分 (负MAE): {grid_lasso.best_score_:.4f}")
    print(f"  最优 Lasso 参数 (alpha): {grid_lasso.best_params_}")
    
    # 获取最优模型 (这个 pipeline 已经是训练好的)
    best_lasso_pipeline = grid_lasso.best_estimator_
    
except Exception as e:
    print(f"  GridSearchCV 训练时出错: {e}")
    exit()

# --- 8. 在训练集上评估最优模型 ---
print("\n--- 在完整训练集上评估最优 Lasso 模型 ---")
try:
    y_train_pred = best_lasso_pipeline.predict(X_train)
    
    # 计算指标
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    mae_train = mean_absolute_error(y_train, y_train_pred)
    medae_train = median_absolute_error(y_train, y_train_pred)
    
    print(f"  训练集 RMSE:  {rmse_train:.4f}")
    print(f"  训练集 MAE:    {mae_train:.4f}") # 这个 MAE 应该接近 CV 中的最优 MAE 的绝对值
    print(f"  训练集 MedAE: {medae_train:.4f}")
    
except Exception as e:
    print(f"  评估训练集时出错: {e}")

# --- 9. 在测试集上进行预测 ---
print("\n--- 在测试集上进行预测 ---")
try:
    predictions = best_lasso_pipeline.predict(X_test)
    print("  预测完成。")
    
    # 可选: 确保预测值为非负数
    # predictions = np.clip(predictions, a_min=0, a_max=None)
    # print("  已将负数预测值修正为 0。")

except Exception as e:
    print(f"  在测试集上预测时出错: {e}")
    exit()

# --- 10. 创建并保存提交文件 ---
print(f"\n--- 创建并保存提交文件到 '{OUTPUT_FOLDER}' ---")

submission_df = pd.DataFrame({
    ID_COLUMN: test_ids,
    'PredictedPrice': predictions 
})

# 确保 ID 是整数
submission_df[ID_COLUMN] = submission_df[ID_COLUMN].astype(int)

# 按 ID 排序
submission_df = submission_df.sort_values(by=ID_COLUMN)

try:
    output_path = os.path.join(OUTPUT_FOLDER, OUTPUT_FILE)
    submission_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✓ 提交文件已成功保存: {output_path}")
    print(f"  总预测条数: {len(submission_df)}")
    print("\n提交文件预览 (前5行):")
    print(submission_df.head())
except Exception as e:
    print(f"✗ 保存提交文件时出错: {e}")

print("\n--- Lasso (6折 CV) 预测流程完成 ---")

输出文件夹 'Lasso_Prediction_Result' 已创建或已存在。

--- 从 'Feature_Selected_Data' 加载 Price 数据 ---
Price 数据加载成功。
  训练集形状: (103871, 155)
  测试集形状: (34017, 155)

--- 准备训练和测试数据 ---
  警告: 训练集发现非数值列，将移除: ['楼层_中楼层', '楼层_低楼层', '楼层_地下室', '楼层_底层', '楼层_顶层', '楼层_高楼层', '环线_三至四环', '环线_中环至外环', '环线_二环内', '环线_二至三环', '环线_五至六环', '环线_六环外', '环线_内环内', '环线_内环至中环', '环线_内环至外环', '环线_四至五环', '环线_外环外', '环线_无环线', '电梯_无', '电梯_有', '电梯_未知', '建筑结构_未知结构', '建筑结构_框架结构', '建筑结构_混合结构', '建筑结构_砖木结构', '建筑结构_砖混结构', '建筑结构_钢混结构', '建筑结构_钢结构', '装修_精装', '装修_简装', '装修_毛坯', '装修_其他', '装修_未知', '装修_精装.1', '装修_简装.1', '装修_毛坯.1', '装修_其他.1', '装修_未知.1', '别墅_双拼', '别墅_叠拼', '别墅_独栋', '别墅_联排', '别墅_非别墅', '产权所属_共有', '产权所属_非共有']
  警告: 测试集发现非数值列，将移除: ['楼层_中楼层', '楼层_低楼层', '楼层_地下室', '楼层_底层', '楼层_顶层', '楼层_高楼层', '环线_三至四环', '环线_中环至外环', '环线_二环内', '环线_二至三环', '环线_五至六环', '环线_六环外', '环线_内环内', '环线_内环至中环', '环线_内环至外环', '环线_四至五环', '环线_外环外', '环线_无环线', '电梯_无', '电梯_有', '电梯_未知', '建筑结构_未知结构', '建筑结构_框架结构', '建筑结构_混合结构', '建筑结构_砖木结构', '建筑结构_砖混结构', '建筑结构_钢混结构', '建筑结构_钢结构', '装修_精装', '装修_简装

c:\Users\32854\anaconda3\envs\garch_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.019e+16, tolerance: 6.664e+13
  model = cd_fast.enet_coordinate_descent(



  GridSearchCV 完成。
  最优 Lasso 交叉验证得分 (负MAE): -934325.4740
  最优 Lasso 参数 (alpha): {'model__alpha': 100.0}

--- 在完整训练集上评估最优 Lasso 模型 ---
  训练集 RMSE:  1564370.8916
  训练集 MAE:    933018.2270
  训练集 MedAE: 624141.8045

--- 在测试集上进行预测 ---
  预测完成。

--- 创建并保存提交文件到 'Lasso_Prediction_Result' ---
✓ 提交文件已成功保存: Lasso_Prediction_Result\submission_lasso_6fold_mae.csv
  总预测条数: 34017

提交文件预览 (前5行):
        ID  PredictedPrice
0  1000000    1.007367e+07
1  1000001    4.357744e+06
2  1000002    6.577985e+06
3  1000003    4.148607e+06
4  1000004    6.523672e+06

--- Lasso (6折 CV) 预测流程完成 ---
